In [1]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class ConditionalBatchNorm2d(pl.LightningModule):

    def __init__(self, input_shape: int, num_classes: int):
        """
        Creates 2 type of variables for y is None and y is not None
        :param input_shape: it is x.shape[-1], we need to know it in at compile-time, I think we could guess it in debugging
        :param num_classes: NUmber of outputting features
        """
        super().__init__()

        self.input_shape = input_shape
        self.num_classes = num_classes

        self.beta1 = torch.zeros(size=[input_shape], requires_grad=True)
        self.gamma1 = torch.ones(size=[input_shape], requires_grad=True)

        self.beta2 = torch.zeros(size=[num_classes, input_shape], requires_grad=True)
        self.gamma2 = torch.ones(size=[num_classes, input_shape], requires_grad=True)

    def forward(self, x, y=None):
        if y is not None:
            # self.beta1 = torch.reshape(torch.index_select(self.beta2, 0, y), [-1, 1, 1, self.input_shape])
            # self.gamma1 = torch.reshape(torch.index_select(self.gamma2, 0, y), [-1, 1, 1, self.input_shape])
            print("BN y.shape =", y.shape, ", type(y) =", type(y))

            embedding_weight = nn.Embedding.from_pretrained(self.beta2)
            embedding_gamma = nn.Embedding.from_pretrained(self.gamma2)

            self.beta2 = torch.reshape(embedding_weight(y), [-1, self.input_shape])
            self.gamma2 = torch.reshape(embedding_gamma(y), [-1, self.input_shape])

        running_mean, running_var = torch.mean(x, dim=[0, 2, 3]), torch.var(x, dim=[0, 2, 3])

        print("God save me")
        print("x shape:", x.size())
        print("mean shape:", running_mean.size())
        print("var shape", running_var.size())
        print("beta shape", self.beta1.size())
        print("gamma shape", self.gamma1.size())

        normalized = F.batch_norm(x, running_mean, running_var, self.beta1, self.gamma1, eps=1e-3,
                                  momentum=0.5)

        return normalized


In [3]:
# x = torch.randn(size=[5, 3, 64, 64], requires_grad=True).to('cuda')
# y = torch.randint(low=0, high=5, size=[32]).to('cuda')

# bn = ConditionalBatchNorm2d(input_shape=3, num_classes=10)

# output = bn(x, y)

# print(output.size)

# loss = torch.nn.L1Loss()(output, temp)
# loss.backward()

In [5]:
temp = torch.randn(size=[5, 3, 64, 64], requires_grad=True)

y = torch.randint(0, 10, size=[32])

bn = ConditionalBatchNorm2d(input_shape=3, num_classes=10)

output = bn(temp, y)

print("output size:", output.size())

loss = torch.nn.L1Loss()(output, temp)
loss.backward()

BN y.shape = torch.Size([32]) , type(y) = <class 'torch.Tensor'>
God save me
x shape: torch.Size([5, 3, 64, 64])
mean shape: torch.Size([3])
var shape torch.Size([3])
beta shape torch.Size([3])
gamma shape torch.Size([3])


RuntimeError: the derivative for 'running_mean' is not implemented